A man looking at a portrait says:

> I am an only child, and the portrait's father is my father's son.

Who is this a portrait of ?

Allowed primitives:
* As many boolean variables as you want
* P is true
* P is false
* Any boolean operations (and, or, etc.)
* P $\Rightarrow$ P'
* Any composition of those (e.g. $P \Rightarrow (P' \Rightarrow P'')$)


https://archive.org/details/WhatIsTheNameOfThisBook/page/n23/mode/2up  page 20

In [ ]:
!pip install ortools

In [ ]:
from ortools.sat.python import cp_model
# Derived from https://stackoverflow.com/questions/58934609/obtain-list-of-sat-solutions-from-ortools
class VarArraySolutionCollector(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.solution_list = []

    def on_solution_callback(self):
        self.solution_list.append([self.Value(v) for v in self.__variables])
    
    def __repr__(self):
      print(f"There are {len(self.solution_list)} solutions of {len(self.__variables)} variables.")
      true = []
      false = []
      undecided = []
      for i, v in enumerate(self.__variables):
        values = set(l[i] for l in self.solution_list)
        if values == {1}:
          true += [f'{v}']
        elif values == {0}:
          false += [f'{v}']
        elif values == {0, 1}:
          undecided += [f'{v}']
        else:
          raise ValueError(f"WTF {v} {values}")
      answer =  f'*****************\nTrue statements ({len(true)}): \n'
      answer += '\n'.join(true)+'\n'
      answer += f'*****************\nUndecided statements ({len(undecided)}): \n'
      answer += '\n'.join(undecided)+'\n'
      answer += f'*****************\nFalse statements ({len(false)}): \n'
      answer += '\n'.join(false)+'\n'
      return answer


In [1]:
def AandBimpliesC(model, A, B, C):
  """Make the model understand that (A and B) => C.
  
  We use the fact that D=>C == Not(D) or C,
  
  and De Morgan's rule: Not(A and B) == Not(A) or Not(B)."""
  model.AddBoolOr([A.Not(), B.Not(), C])
  return model

def AandBequivC(model, A, B, C):
  """Make the model understand that (A and B) <=> C."""
  AandBimpliesC(model, A, B, C)
  model.AddImplication(C, A)
  model.AddImplication(C, B)
  return model

def AequivB(model, A, B):
  """Make the model understand that A <=> B"""
  model.AddImplication(A, B)
  model.AddImplication(B, A)
  return model

In [ ]:
model = cp_model.CpModel()
CHARACTERS = ['Speaker', 'Portrait', "Speaker's father", "Portrait's father",
              "Speaker's father's son"]
relationships = {}

def child(a, b, sentence=False):
  """Return the bool var for 'a is the child of b'.

  If sentence is True, return only the string."""
  _sentence = f"The {a} is a child of the {b}."
  if sentence:
    return _sentence
  return relationships[_sentence]

def _is(a, b, sentence=False):
  """Return both the bool var for 'a is b'.
  
  If sentence is True, return only the string."""
  _sentence = f"The {a} and the {b} are the same person."
  if sentence:
    return _sentence
  return relationships[_sentence]

for x in CHARACTERS:
  for y in CHARACTERS:
    sentence = child(x, y, sentence=True)
    relationships[sentence] = model.NewBoolVar(sentence)
    sentence = _is(x, y, sentence=True)
    relationships[sentence] = model.NewBoolVar(sentence)


# Identity
for x in CHARACTERS:
  model.Add(_is(x, x) == 1)


for x in CHARACTERS:
  for y in CHARACTERS:
    # If x is y's child, 
    # x and y can not be the same person
    model.AddImplication(child( x,y), _is(x, y).Not())
    # and conversely
#    model.AddImplication(_is(x, y), child(x, y).Not())
    # And y can not be the child of x
    model.AddImplication(child(x, y), child(y, x).Not())
    # Also, _is is symetrical
    model.AddImplication(_is(x, y), _is(y, x))
#    model.AddImplication(_is(x, y).Not(), _is(y, x).Not())
    # And transitive
    for z in CHARACTERS:
      AandBimpliesC(model,
                  A=_is(x, y),
                  B=_is(y, z),
                  C=_is(x, z))
      # And is x and y are the same person, they have the same children
      AandBimpliesC(model,
                    A=_is(x, y),
                    B=child(z, x),
                    C=child(z, y))
    
# Father son relationships
model.Add(child("Speaker", "Speaker's father") == 1)
model.Add(child("Portrait", "Portrait's father") == 1)
model.Add(child("Speaker's father's son", "Speaker's father") == 1)

# Problem statement: identity
model.Add(_is("Portrait's father", "Speaker's father's son") == 1)

# Problem statement: Speaker is an only child
for x in CHARACTERS:
  # Speaker is a child of x
  for y in CHARACTERS:
    #  Speaker and y are NOT the same person
    # => y is NOT a child of x
    AandBimpliesC(model, 
                A=child('Speaker', x),
                B=_is('Speaker', y).Not(),
                C=child(y, x).Not())